# Модуль А. Предобработка количественных данных

Подключим стандартные библиотеки для работы с даннами, а так же отключим предупреждения и добавим автоматическую отрисовку графиков

In [24]:
pip install pmdarima

Note: you may need to restart the kernel to use updated packages.


In [27]:
import pmdarima

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [2]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

## Сбор данных

In [3]:
df = pd.read_csv("data/Microsoft_Stock.csv", index_col="Date", parse_dates=True)
df.head(10)

,Open,High,Low,Close,Volume
Date,,,,,
2015-04-01 16:00:00,40.60,40.76,40.31,40.72,36865322
2015-04-02 16:00:00,40.66,40.74,40.12,40.29,37487476
2015-04-06 16:00:00,40.34,41.78,40.18,41.55,39223692
2015-04-07 16:00:00,41.61,41.91,41.31,41.53,28809375
2015-04-08 16:00:00,41.48,41.69,41.04,41.42,24753438
2015-04-09 16:00:00,41.25,41.62,41.25,41.48,25723861
2015-04-10 16:00:00,41.63,41.95,41.41,41.72,28022002
2015-04-13 16:00:00,41.40,42.06,41.39,41.76,30276692
2015-04-14 16:00:00,41.80,42.03,41.39,41.65,24244382


Выведем и просмотрим первые записи

In [4]:
df.head()

,Open,High,Low,Close,Volume
Date,,,,,
2015-04-01 16:00:00,40.60,40.76,40.31,40.72,36865322
2015-04-02 16:00:00,40.66,40.74,40.12,40.29,37487476
2015-04-06 16:00:00,40.34,41.78,40.18,41.55,39223692
2015-04-07 16:00:00,41.61,41.91,41.31,41.53,28809375
2015-04-08 16:00:00,41.48,41.69,41.04,41.42,24753438


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1511 entries, 2015-04-01 16:00:00 to 2021-03-31 16:00:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    1511 non-null   float64
 1   High    1511 non-null   float64
 2   Low     1511 non-null   float64
 3   Close   1511 non-null   float64
 4   Volume  1511 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 70.8 KB


# Модуль Б. Анализ количественных данных

## Анализ основных свойств данных

Посмотрим основные статистические характеристики всех признаков

In [6]:
df.describe(include="all")

,Open,High,Low,Close,Volume
count,1511.000000,1511.000000,1511.000000,1511.000000,1.511000e+03
mean,107.385976,108.437472,106.294533,107.422091,3.019863e+07
std,56.691333,57.382276,55.977155,56.702299,1.425266e+07
min,40.340000,40.740000,39.720000,40.290000,1.016120e+05
25%,57.860000,58.060000,57.420000,57.855000,2.136213e+07
50%,93.990000,95.100000,92.920000,93.860000,2.662962e+07
75%,139.440000,140.325000,137.825000,138.965000,3.431962e+07
max,245.030000,246.130000,242.920000,244.990000,1.352271e+08


Судя по статистическим характеристикам можно предположить, что имеется корреляция между признаками Open, High, Low и Close, так как их показатели очень похожи. Однако Open и Close всегда распологаются между Low и High или на том же уровне, да и показатели зачастую близки друг к другу, что объясняет похожие значения.

In [7]:
df.hist(figsize=(12, 12));

Признаки Open, Close, Low, High имею практически одинаковые графики. Распределение признака Volume издали похоже на нормальное, проверим его на нормальность распределения.

Импортируем библиотеки для определения нормальности признаков

In [8]:
from scipy.stats import shapiro, normaltest
from statsmodels.graphics.gofplots import qqplot
from matplotlib import pyplot

In [9]:
qqplot(df.Close, line="s")
print(shapiro(df.Close))
print(normaltest(df.Close))

ShapiroResult(statistic=np.float64(0.885994186168038), pvalue=np.float64(5.74751543607433e-32))
NormaltestResult(statistic=np.float64(159.04310592710507), pvalue=np.float64(2.9122479991140215e-35))


Имеются большие хвосты и отклонения на qqplotНе, а так же смотря на высокое значение stat, p-значение очень низкое ( < 0.05), поэтому распределение обоих признаков нельзя назвать нормальным.

Построим pairplot для визуального определения зависимостей

In [10]:
sns.pairplot(df);

Имеется линейная зависимость между признаками Open, Close, Low, High. Зависимостей между признаком Volume и остальными не наблюдается.

In [11]:
plt.figure(figsize=(10, 7))
sns.heatmap(df.corr(), annot=True, fmt="0.2f", vmin=-1, cmap="crest");

Как видно на корреляционной карте имеется прямая зависимость межнду основными признаками, хотя, не смотря на близость значений ожидалось увидеть корреляцию около 0.99, но не 1, что удивительно.

Признак Volume практически не коррелирует.

Построим график цен закрытия в соответствии со временем

In [12]:
ax = df.Close.plot(figsize=(12, 6), legend=False)
ax.set(title="График закрытия цен акций Microsoft", xlabel="Дата", ylabel="Стоимость");

In [13]:
ax = df.Close.plot(figsize=(12, 6))
ax.plot(df.Close.rolling(window=365).mean(), color="green")
ax.set(title="График закрытия цен акций Microsoft", xlabel="Дата", ylabel="Стоимость");

In [14]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [15]:
decompose = seasonal_decompose(df.Close, period=365)
decompose.plot();

Большую часть времени тренд восходящий. Имеются сильные сезонные колебания, однако этого не ожидалось, так как цены акций на рынке не столь сильно зависят от месяца и года.

Так же имееются сезонные колебания, особенно сильные в конце временной линии.

Проверим временной ряд на нестационарность

In [16]:
from statsmodels.tsa.stattools import adfuller

In [17]:
adfuller(df.Close)[1]

np.float64(0.9982158366942122)

Как видно, p-value больше 0.05, поэтому отвергнуть нулевую теорию о нестационарности нельзя. Процесс нестанионарный

Вычислиа автокорреляцию с лагом в 365 дней

In [18]:
np.corrcoef(df.Close[:-365], df.Close[365:])[0, 1]

np.float64(0.9734576172161554)

## Моделирование и построение прогноза

Разделим набор данных на обучающую и тестовую выборку

In [19]:
train = df[:-260].Close
test = df[-260:].Close

In [20]:
plt.figure(figsize=(12, 6))
plt.plot(train, label="Обучающая выборка")
plt.plot(test, label="Тестовая выборка", color="green")
plt.legend()

plt.xlabel("Дата")
plt.ylabel("Цена на момент закрытия")
plt.title("Разбиение данных");

In [21]:
from pmdarima import auto_arima

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
model = auto_arima(train, seasonial=True, m=30, trace=True)

### SARIMAX

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
sarima_model = SARIMAX(train, order=(1, 1, 0), seasonal_order=(0, 0, 1, 30))
sarima_result = sarima_model.fit();

In [ ]:
test.head()

In [ ]:
forecast = sarima_result.forecast(steps=365)
forecast_index = pd.date_range("2020-03-20", periods=365, freq="B")
forecast.index = forecast_index

In [ ]:
print(train[-1])
print(forecast)
print(forecast_index)

In [ ]:
plt.plot(train)
plt.plot(test)
plt.plot(forecast, label="Прогноз", color="red")
plt.legend()
plt.show()

### Support Vector Regression